In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.8 MB/s eta 0:00:00


In this project, the fine-tuning of a GPT-2 model will be demonstrated to enhance its ability to infer the sentiments of tweets. To improve its performance, fine-tuning will be leveraged by training the pre-trained GPT-2 model from Hugging Face using a dataset of tweets paired with their corresponding sentiments. Below is a concise outline of the process:

Step 1: Selection of a Pre-Trained Model and Dataset:

The first step in the fine-tuning process involves the selection of a pre-trained model. For this task, GPT-2 will be utilized as the base model, along with a suitable dataset that will facilitate effective fine-tuning.


Step 2: Loading the Dataset:

With the model selected, the next critical step involves the acquisition of high-quality data for fine-tuning. This is where the Hugging Face Datasets library is recognized as invaluable. For this project, the Hugging Face Datasets library will be utilized to import a dataset that categorizes tweets based on their sentiments: Positive, Neutral, or Negative. This structured dataset will serve as the foundation for enhancing the model's sentiment analysis capabilities.



In [3]:
from datasets import load_dataset
import pandas as pd
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

dataset = load_dataset("mteb/tweet_sentiment_extraction")

df = pd.DataFrame(dataset['train'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/3.63M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/465k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27481 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3534 [00:00<?, ? examples/s]

Step 3: Tokenizing the Dataset:

Having acquired the dataset, the next essential step involves the preparation of the data for the model by utilizing a tokenizer. Since large language models (LLMs) operate on tokens, a tokenizer is necessary for appropriate processing of the dataset.
To efficiently tokenize the entire dataset, the Datasets library's map method will be employed, allowing for the application of a preprocessing function across the complete dataset in one step. Therefore, the focus of this step is on loading a pre-trained tokenizer and tokenizing the dataset, ensuring that it is ready for fine-tuning.


In [4]:
from transformers import GPT2Tokenizer
dataset = load_dataset("mteb/tweet_sentiment_extraction")

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

To optimize processing requirements, a smaller subset of the full dataset will be created for fine-tuning the model. This approach allows for more efficient training while maintaining effective evaluation.
The training set will be utilized for fine-tuning the model, while the testing set will serve as a benchmark for evaluating its performance. This division ensures that the model's capabilities can be assessed on unseen data, thereby enhancing the overall robustness of the fine-tuning process.



In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Step 4: Initializing the Base Model:

The next step involves loading the base model and configuring it to recognize the specific number of labels relevant to this task. Based on the sentiment analysis dataset for tweets, three distinct labels Positive, Neutral, and Negative are identified.
By initializing the model with this information, it is ensured that the model is adequately prepared for the fine-tuning process and capable of accurately classifying the sentiment of the input data.



In [6]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step 5: Defining the Evaluation Method:

The Transformers library offers a Trainer class optimized for training models; however, a mechanism for evaluating model performance is not included in the default implementation. Therefore, prior to the commencement of the training process, an evaluation function will need to be provided to the Trainer.
This evaluation function will facilitate the assessment of the model's performance throughout the training process, ensuring that improvements can be monitored and quantified effectively.



In [7]:


metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

Step 6: Fine-Tuning Using the Trainer Method:

The final step involves the configuration of the training arguments and the initiation of the training process. The Transformers library provides the Trainer class, which supports a comprehensive array of training options and features, including logging, gradient accumulation, and mixed precision.
Initially, the training arguments will be defined along with the evaluation strategy. Once all parameters are established, the model can be trained effortlessly using the train() command, streamlining the fine-tuning process.



In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
   output_dir="test_trainer",

   per_device_train_batch_size=1,
   per_device_eval_batch_size=1,
   gradient_accumulation_steps=4
   )


trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=small_train_dataset,
   eval_dataset=small_eval_dataset,
   compute_metrics=compute_metrics,

)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


Model Evaluation:

Following the training process, the model’s performance will be evaluated on a validation or test set. The Trainer class includes an evaluate method that efficiently manages this evaluation. By utilizing this built-in functionality, a comprehensive assessment of the model's performance can be conducted, providing valuable insights into its effectiveness and accuracy.




In [ ]:

trainer.evaluate()


These are the most basic steps to perform a fine-tuning of any LLM and fine-tuning an LLM is highly computationally demanding.